# Data Preparation
## In this section, we will prepare your data for analysis.

#### Import Libraries

In [ ]:
import pandas as pd
from glob import glob

#### Concatenation of the csv files
##### Our data was split into 6 different csv files. In this section we are concatenating all these files into one single csv file(base_data.csv).

In [13]:
files = glob("Data_Files/Morticd10_part*.csv")

dfs = []
for f in files:
    df = pd.read_csv(f, low_memory=False)
    dfs.append(df)
merged = pd.concat(dfs, ignore_index=True)

merged.shape

##merged.to_csv("Data_Files/base_data.csv", index=False)

(5143931, 39)

In [18]:
merged['List'].unique()

array(['101', '103', '104', '10M', 'UE1', 101, 103, 104], dtype=object)

In [19]:
merged["List"].value_counts()

List
104    2920684
104    1790218
103     225457
103     158080
10M      18114
101      17126
101      14000
UE1        252
Name: count, dtype: int64

#### Deletion of unnecessary rows
##### In this section we are deleting all the rows that are not necessary for our analysis.

In [16]:
data = merged[(merged["List"] == 103) | (merged["List"] == '103')]

data.to_csv("Data_Files/base_data.csv", index=False)

#### Lookup Country Names
##### In this section we are looking up the country names based on the country codes provided in the dataset (country_codes.csv).

In [20]:
codes = pd.read_csv("codes_Files\country_codes.csv", encoding="utf-8-sig")
mortality = pd.read_csv(r"Data_Files\base_data.csv", encoding="utf-8-sig")
merged = mortality.merge(codes, left_on="Country", right_on="country", how="left")
merged["Country"] = merged["name"]
merged = merged.drop(columns=["country", "name"])
merged.to_csv("mortality_with_country_names.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\benma\AppData\Local\Temp\ipykernel_30340\2146119423.py:1: SyntaxWarning: invalid escape sequence '\c'
  codes = pd.read_csv("codes_Files\country_codes.csv", encoding="utf-8-sig")
C:\Users\benma\AppData\Local\Temp\ipykernel_30340\2146119423.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  mortality = pd.read_csv(r"Data_Files\base_data.csv", encoding="utf-8-sig")
